In [117]:
import pandas as pd
import numpy as np
import os

In [118]:
pwd

'/Users/chrissoria/Documents/Research/BICS_Political_Polarization'

In [119]:
os.chdir('/Users/chrissoria/Documents/Research/BICS_Political_Polarization/')

some ZIP Codes, such as "unique" ZIPs that are often tied to an internal mail-routing mechanism (such as UC Berkeley which has its own internal ZIP code and mail-routing system) will have very low ratios). \
As of 12/23 I'm realzing that I'll need to start with a complete list of all ZIP codes (from USPS) before I start this so I can get a good sense of completeness. 

Below, I'm reading in that full list of ZIP codes

In [120]:
ZIP_to_CD_df = pd.read_excel('Data/ZIP_CD_062020.xlsx')

ZIP_to_CD_df = ZIP_to_CD_df.rename(columns={
    'RES_RATIO': 'CD_RES_RATIO',
    'BUS_RATIO': 'CD_BUS_RATIO',
    'OTH_RATIO': 'CD_OTH_RATIO',
    'TOT_RATIO': 'CD_TOT_RATIO'
})

ZIP_to_CD_df['ZIP'] = ZIP_to_CD_df['ZIP'].astype(str).str.zfill(5)

In [121]:
col_types = {
    'DELIVERY ZIPCODE': str
}

FULL_ZIPS = pd.read_excel('Data/Full_Zips.xlsx', dtype=col_types)

print(len(FULL_ZIPS))

FULL_ZIPS = FULL_ZIPS[['DELIVERY ZIPCODE','PHYSICAL STATE','TYPE']]

FULL_ZIPS = FULL_ZIPS.rename(columns={
    'DELIVERY ZIPCODE': 'ZIP',
    'PHYSICAL STATE': 'USPS_STATE'
})

#removing ZIP codes that aren't in US main
FULL_ZIPS = FULL_ZIPS.drop_duplicates(subset='ZIP', keep='first')
FULL_ZIPS['ZIP'] = FULL_ZIPS['ZIP'].astype(str).str.zfill(5)
FULL_ZIPS = FULL_ZIPS[pd.notna(FULL_ZIPS['USPS_STATE'])].reset_index(drop=True)
exclude_states = ["VI", "PR", "MH", "MP", "FM", "GU"]
FULL_ZIPS = FULL_ZIPS[~FULL_ZIPS['USPS_STATE'].isin(exclude_states)].reset_index(drop=True)
FULL_ZIPS = FULL_ZIPS[~FULL_ZIPS['TYPE'].isin(["Other", "Unique"])].reset_index(drop=True)

print(len(FULL_ZIPS))
print(FULL_ZIPS.head())

49649
37714
     ZIP USPS_STATE             TYPE
0  01001         MA  Standard/PO BOX
1  01002         MA  Standard/PO BOX
2  01004         MA  Standard/PO BOX
3  01005         MA  Standard/PO BOX
4  01007         MA  Standard/PO BOX


/Users/chrissoria/anaconda3/lib/python3.11/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


After merging, I find that many of the Alaska ZIP codes aren't getting matched. Since there's only one CD in Alaska, I'll fill in any Alaska state ZIP with 0200. \
This data will not match PO boxes. Most non matches are coming from PO boxes.

In [122]:
ZIP_FEATURES = pd.merge(FULL_ZIPS, ZIP_to_CD_df, on = "ZIP", how = "left") #merging to full zips here

ZIP_FEATURES['CD'] = ZIP_FEATURES.apply(lambda row: "0200" if row['USPS_STATE'] == "AK" else row['CD'], axis = 1)
ZIP_FEATURES['CD_TOT_RATIO'] = ZIP_FEATURES.apply(lambda row: 1.0 if row['USPS_STATE'] == "AK" else row['CD_TOT_RATIO'], axis = 1)

ZIP_FEATURES['CD'] = ZIP_FEATURES.apply(lambda row: "4212" if row['ZIP'] == "15253" else row['CD'],
                                       axis=1)

ZIP_FEATURES

,ZIP,USPS_STATE,TYPE,CD,CD_RES_RATIO,CD_BUS_RATIO,CD_OTH_RATIO,CD_TOT_RATIO
0,01001,MA,Standard/PO BOX,2501,1.000000,1.0,1.0,1.000000
1,01002,MA,Standard/PO BOX,2502,1.000000,1.0,1.0,1.000000
2,01004,MA,Standard/PO BOX,2502,1.000000,1.0,1.0,1.000000
3,01005,MA,Standard/PO BOX,2502,1.000000,1.0,1.0,1.000000
4,01007,MA,Standard/PO BOX,2502,0.998118,1.0,1.0,0.998217
...,...,...,...,...,...,...,...,...
44276,99925,AK,Standard/PO BOX,0200,0.000000,0.0,1.0,1.000000
44277,99926,AK,Standard/PO BOX,0200,0.000000,0.0,1.0,1.000000
44278,99927,AK,Standard/PO BOX,0200,0.000000,0.0,1.0,1.000000
44279,99928,AK,Standard/PO BOX,0200,0.000000,0.0,1.0,1.000000


For now, I will assign a ZIP code to whatever whatever County it falls mostly under residentially \
This will NOT include DC simply because DC doesn't have a CD \
IN A FUTURE VERSION, I WILL HAVE TO START WITH A LIST OF ZIP CODES THAT INCLUDE DISTRICT OF COLUMBIA'S (WHICH I  HAVE ABOVE)

I'm running this as an outer match because then I can keep zip codes from the right side that aren't in the left side

In [123]:
col_types = {
    'COUNTY': str,
    'ZIP': str
}

ZIP_to_County_df = pd.read_excel('Data/ZIP_COUNTY_062020_HUD.xlsx', dtype=col_types)
ZIP_to_County_df['COUNTY'] = ZIP_to_County_df['COUNTY'].astype(str).str.zfill(5)
ZIP_to_County_df['ZIP'] = ZIP_to_County_df['ZIP'].astype(str).str.zfill(5)
print(ZIP_to_County_df.head())
ZIP_FEATURES = pd.merge(ZIP_FEATURES, ZIP_to_County_df, how ='outer', on = "ZIP")

# Sort the DataFrame by RES_RATIO in descending order
ZIP_FEATURES = ZIP_FEATURES.sort_values(by='RES_RATIO', ascending=False)
ZIP_FEATURES = ZIP_FEATURES.drop_duplicates(subset='ZIP', keep='first')

ZIP_to_County_df.reset_index(drop=True, inplace=True)

ZIP_to_County_df = ZIP_to_County_df.rename(columns={
    'RES_RATIO': 'COUNTY_RES_RATIO',
    'BUS_RATIO': 'COUNTY_BUS_RATIO',
    'OTH_RATIO': 'COUNTY_OTH_RATIO',
    'TOT_RATIO': 'COUNTY_TOT_RATIO'
})

print(ZIP_FEATURES.head())

print(ZIP_FEATURES['COUNTY'].isna().sum()) #high level of match
print(ZIP_FEATURES['COUNTY'].isna().mean() * 100)

     ZIP COUNTY  RES_RATIO  BUS_RATIO  OTH_RATIO  TOT_RATIO
0  00501  36103   0.000000   1.000000   0.000000   1.000000
1  00601  72113   0.160758   0.199017   0.128834   0.162397
2  00601  72001   0.839242   0.800983   0.871166   0.837603
3  00602  72003   1.000000   0.998801   1.000000   0.999919
4  00602  72005   0.000000   0.001199   0.000000   0.000081
         ZIP USPS_STATE             TYPE    CD  CD_RES_RATIO  CD_BUS_RATIO  \
0      01001         MA  Standard/PO BOX  2501           1.0           1.0   
28914  45729         OH  Standard/PO BOX  3906           1.0           1.0   
28955  45771         OH  Standard/PO BOX  3906           1.0           1.0   
28954  45770         OH  Standard/PO BOX  3906           1.0           1.0   
28953  45769         OH  Standard/PO BOX  3906           1.0           1.0   

       CD_OTH_RATIO  CD_TOT_RATIO COUNTY  RES_RATIO  BUS_RATIO  OTH_RATIO  \
0               1.0           1.0  25013        1.0        1.0        1.0   
28914           1

I'm realizing I'll have to merge the school district data onto ZIP_to_County_df (give that there's some zip codes that don't have a school district associated) \
Yet, the issue of some ZIP codes falling through the cracks will still be there. \
I'll need something that goes from county and matches to all ZIP codes within OR \
Something that matches ZIP codes directly to the nearest school district (IDEAL). \
For now (12/26/23) this variable is not complete. 

Instead, I'll match the ZIP code to a census tract THEN match tract onto school district

In [124]:
col_types = {
    'TRACT': str,
    'ZIP': str
}

ZIP_to_Tract = pd.read_excel('data/ZIP_TRACT_062020.xlsx', dtype=col_types)

ZIP_to_Tract = ZIP_to_Tract.sort_values(by='RES_RATIO', ascending=False)
ZIP_to_Tract = ZIP_to_Tract.drop_duplicates(subset='ZIP', keep='first')

ZIP_to_Tract.reset_index(drop=True, inplace=True)

ZIP_FEATURES = pd.merge(ZIP_FEATURES, ZIP_to_Tract, on = "ZIP", how = "left")

ZIP_FEATURES

,ZIP,USPS_STATE,TYPE,CD,CD_RES_RATIO,CD_BUS_RATIO,CD_OTH_RATIO,CD_TOT_RATIO,COUNTY,RES_RATIO_x,BUS_RATIO_x,OTH_RATIO_x,TOT_RATIO_x,TRACT,RES_RATIO_y,BUS_RATIO_y,OTH_RATIO_y,TOT_RATIO_y
0,01001,MA,Standard/PO BOX,2501,1.0,1.0,1.0,1.0,25013,1.0,1.0,1.0,1.0,25013813209,0.404431,0.442809,0.250000,0.403828
1,45729,OH,Standard/PO BOX,3906,1.0,1.0,1.0,1.0,39167,1.0,1.0,1.0,1.0,39167021700,0.752549,0.533333,1.000000,0.749772
2,45771,OH,Standard/PO BOX,3906,1.0,1.0,1.0,1.0,39105,1.0,1.0,1.0,1.0,39105964600,0.793269,0.693548,0.692308,0.788959
3,45770,OH,Standard/PO BOX,3906,1.0,1.0,0.0,1.0,39105,1.0,1.0,0.0,1.0,39105964600,0.923077,1.000000,0.000000,0.924791
4,45769,OH,Standard/PO BOX,3906,1.0,1.0,1.0,1.0,39105,1.0,1.0,1.0,1.0,39105964200,0.475592,0.139276,0.127660,0.440522
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39916,99711,AK,Standard/PO BOX,0200,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39917,99716,AK,Standard/PO BOX,0200,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39918,99802,AK,Standard/PO BOX,0200,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39919,99803,AK,Standard/PO BOX,0200,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Below, I'll read in the county-school district level data

col_types = {
    'LEAID': str,
    'STCOUNTY': str
}

County_to_District = pd.read_excel('data/grf20_lea_county.xlsx', dtype=col_types)
County_to_District = County_to_District.sort_values(by='LANDAREA', ascending=False)
County_to_District = County_to_District.drop_duplicates(subset='STCOUNTY', keep='first')

County_to_District = County_to_District.rename(columns={"STCOUNTY": "COUNTY"})

County_to_District = County_to_District[['NAME_LEA20','COUNTY','LEAID']]

ZIP_FEATURES = pd.merge(ZIP_FEATURES, County_to_District, on = "COUNTY", how = "left")

ZIP_FEATURES.head()

We also want to add data on mask mandates at the county-level in order to catpure "given that a mask mandatte was or wasn't implementd...partisans contact is..."

Next, I'll clean up the data so that we keep only the desired columns, differentiate between the census tract (higher resolution) school district match and the county-level (lower resolution) match. Lastly, I'll create one column for school district based on both of these types of matches

ZIP_FEATURES = ZIP_FEATURES.rename(columns = {
                                   'NAME_LEA20': 'COUNTY_SCH_DISTRICT',
                                   'LEAID': 'COUNTY_LEAID'})

ZIP_FEATURES.head()

NOTE: The school closure data provides best coverage for months after september 2020 to May 2021

schools_closures = pd.read_csv('data/District_Monthly_Shares_03.08.23.csv',dtype=str)

columns_to_keep = ['NCESDistrictID','month','share_inperson','DistrictName']

schools_closures = schools_closures[columns_to_keep]

schools_closures['LEAID'] = schools_closures['NCESDistrictID'].apply(lambda x: '00' + x if len(x) == 5 else x)

schools_closures['NCESDistrictID'] = schools_closures['LEAID'].apply(lambda x: '0' + x if len(x) == 6 else x)

#for mid 2020 (taking two months in case the first is missing August and September)

schools_closures_m2020 = schools_closures[schools_closures['month'] == "2020m8"].reset_index(drop=True)

schools_closures_m2020 = schools_closures_m2020.rename(columns={
    'share_inperson': 'august20_school_share_inperson'
})

schools_closures_m2020 = schools_closures_m2020[['LEAID','august20_school_share_inperson','DistrictName']]

#now mnth 9
schools_closures_m2020_2 = schools_closures[schools_closures['month'] == "2020m9"].reset_index(drop=True)

schools_closures_m2020_2 = schools_closures_m2020_2.rename(columns={
    'share_inperson': 'september20_school_share_inperson'
})

schools_closures_m2020_2 = schools_closures_m2020_2[['LEAID','september20_school_share_inperson']]

#for month 10 (aggressive capture)

schools_closures_m2020_3 = schools_closures[schools_closures['month'] == "2020m10"].reset_index(drop=True)

schools_closures_m2020_3 = schools_closures_m2020_3.rename(columns={
    'share_inperson': 'october20_school_share_inperson'
})

schools_closures_m2020_3 = schools_closures_m2020_3[['LEAID','october20_school_share_inperson']]

#for late 2020

schools_closures_l2020 = schools_closures[schools_closures['month'] == "2020m12"].reset_index(drop=True)

schools_closures_l2020 = schools_closures_l2020.rename(columns={
    'share_inperson': 'december20_school_share_inperson'
})

schools_closures_l2020 = schools_closures_l2020[['LEAID','december20_school_share_inperson']]


schools_closures_l2020_2 = schools_closures[schools_closures['month'] == "2020m11"].reset_index(drop=True)

schools_closures_l2020_2 = schools_closures_l2020_2.rename(columns={
    'share_inperson': 'november20_school_share_inperson'
})

schools_closures_l2020_2 = schools_closures_l2020_2[['LEAID','november20_school_share_inperson']]


#for mid 2021

schools_closures_m2021 = schools_closures[schools_closures['month'] == "2021m3"].reset_index(drop=True)

schools_closures_m2021 = schools_closures_m2021.rename(columns={
    'share_inperson': 'march21_school_share_inperson'
})

schools_closures_m2021 = schools_closures_m2021[['LEAID','march21_school_share_inperson']]


schools_closures_m2021_2 = schools_closures[schools_closures['month'] == "2021m2"].reset_index(drop=True)

schools_closures_m2021_2 = schools_closures_m2021_2.rename(columns={
    'share_inperson': 'feb21_school_share_inperson'
})

schools_closures_m2021_2 = schools_closures_m2021_2[['LEAID','feb21_school_share_inperson']]

#later mid 2021

schools_closures_lm2021 = schools_closures[schools_closures['month'] == "2021m4"].reset_index(drop=True)

schools_closures_lm2021 = schools_closures_lm2021.rename(columns={
    'share_inperson': 'april21_school_share_inperson'
})

schools_closures_lm2021 = schools_closures_lm2021[['LEAID','april21_school_share_inperson']]


schools_closures_lm2021_2 = schools_closures[schools_closures['month'] == "2021m5"].reset_index(drop=True)

schools_closures_lm2021_2 = schools_closures_lm2021_2.rename(columns={
    'share_inperson': 'may21_school_share_inperson'
})

schools_closures_lm2021_2 = schools_closures_lm2021_2[['LEAID','may21_school_share_inperson']]


Early fall = Aug-Oct
Fall 2020 = Nov-Dec
Spring 2021 = Feb-Mar

schools_closures = schools_closures[~schools_closures['LEAID'].duplicated()]

schools_closures = pd.merge(schools_closures, schools_closures_m2020, how = 'left', on = 'LEAID')
schools_closures = pd.merge(schools_closures, schools_closures_m2020_2, how = 'left', on = 'LEAID')
schools_closures = pd.merge(schools_closures, schools_closures_m2020_3, how = 'left', on = 'LEAID')
schools_closures = pd.merge(schools_closures, schools_closures_l2020_2, how = 'left', on = 'LEAID')
schools_closures = pd.merge(schools_closures, schools_closures_l2020, how = 'left', on = 'LEAID')
schools_closures = pd.merge(schools_closures, schools_closures_m2021_2, how = 'left', on = 'LEAID')
schools_closures = pd.merge(schools_closures, schools_closures_m2021, how = 'left', on = 'LEAID')
schools_closures = pd.merge(schools_closures, schools_closures_lm2021_2, how = 'left', on = 'LEAID')
schools_closures = pd.merge(schools_closures, schools_closures_lm2021, how = 'left', on = 'LEAID')

#aug, sept, oct (2020)
schools_closures['earlyfall2020_school_closures'] = schools_closures.apply(
    lambda row: row['september20_school_share_inperson'] if pd.isna(row['august20_school_share_inperson']) else row['august20_school_share_inperson'], 
    axis=1
)
schools_closures['earlyfall2020_school_closures'] = schools_closures.apply(
    lambda row: row['october20_school_share_inperson'] if pd.isna(row['earlyfall2020_school_closures']) else row['earlyfall2020_school_closures'],
    axis=1
)

#nov, dec (2020)
schools_closures['fall2020_school_closures'] = schools_closures.apply(
    lambda row: row['november20_school_share_inperson'] if pd.isna(row['december20_school_share_inperson']) else row['december20_school_share_inperson'],
    axis=1
)

#feb, march (2021)
schools_closures['earlyspring2021_school_closures'] = schools_closures.apply(
    lambda row: row['feb21_school_share_inperson'] if pd.isna(row['march21_school_share_inperson']) else row['march21_school_share_inperson'], 
    axis=1
)

#april, may (2021)

schools_closures['spring2021_school_closures'] = schools_closures.apply(
    lambda row: row['april21_school_share_inperson'] if pd.isna(row['may21_school_share_inperson']) else row['may21_school_share_inperson'], 
    axis=1
)

schools_closures = schools_closures[['LEAID','earlyfall2020_school_closures','fall2020_school_closures',
                                     'earlyspring2021_school_closures','spring2021_school_closures',]]
schools_closures.to_csv('data/school_closure_merged.csv')

schools_closures

let's take a look at coverage. Based on these missing counts, it looks like the best coverage available is for fall 2020, with early spring 2021 and late spring 2021 having a higher percentage of missingness (almost 10 percent of rows contain missing data)

na_counts = schools_closures[['LEAID', 'earlyfall2020_school_closures', 'fall2020_school_closures',
                              'earlyspring2021_school_closures', 'spring2021_school_closures']].isna().sum()

na_percentage = schools_closures[['LEAID', 'earlyfall2020_school_closures', 'fall2020_school_closures',
                                  'earlyspring2021_school_closures', 'spring2021_school_closures']].isna().mean() * 100


print(na_counts)
print(na_percentage)

ZIP_FEATURES['LEAID'] = ZIP_FEATURES['LEAID'].astype(str)

ZIP_FEATURES['LEAID'] = ZIP_FEATURES['LEAID'].apply(lambda x: '00' + x if len(x) == 5 else x)
ZIP_FEATURES['LEAID'] = ZIP_FEATURES['LEAID'].apply(lambda x: '0' + x if len(x) == 6 else x)

schools_closures['LEAID'] = schools_closures['LEAID'].apply(lambda x: '00' + x if len(x) == 5 else x)
schools_closures['LEAID'] = schools_closures['LEAID'].apply(lambda x: '0' + x if len(x) == 6 else x)

ZIP_FEATURES = pd.merge(ZIP_FEATURES, schools_closures, how = 'left', on = "LEAID")

In terms of ZIP codes, this translates to big percentages. Over 20% of zip codes do not have data on school closures for the spring 2021 semester.

na_counts = ZIP_FEATURES[['LEAID', 'earlyfall2020_school_closures', 'fall2020_school_closures',
                              'earlyspring2021_school_closures', 'spring2021_school_closures']].isna().sum()

na_percentage = ZIP_FEATURES[['LEAID', 'earlyfall2020_school_closures', 'fall2020_school_closures',
                                  'earlyspring2021_school_closures', 'spring2021_school_closures']].isna().mean() * 100


print(na_counts)
print(na_percentage)

I tried to consolidate the 2021 semester category for better coverage, but it made no difference

Maybe it doesn't make sense to have ZIP to CD and ZIP to County in the same dataset. I might make these into two later on, but for now this contains the county under which the ZIP code mostly falls for residential addresses. 

In [125]:
col_types = {
    'FIPS_Code': str,
    'District': str,
    'CD':str
}

CD_df = pd.read_csv('data/House_Reps_District.csv', dtype=col_types)

In [126]:
ZIP_FEATURES['CD'] = ZIP_FEATURES['CD'].astype(str)
CD_df['CD'] = CD_df['CD'].astype(str)


ZIP_FEATURES = pd.merge(ZIP_FEATURES, CD_df, on = "CD", how= "left")

ZIP_FEATURES = ZIP_FEATURES.rename(columns={
    'COUNTY': 'COUNTY_FIPS', #coming from the HUD ZIP to County DF
    'Name': 'CONGRESSPERSON',
    'State': 'STATE',
    'Party': 'CONGRESSPERSON_PARTY'
})

cols_to_move = ZIP_FEATURES.columns[3:5]
df_remaining = ZIP_FEATURES.drop(columns=cols_to_move)
ZIP_FEATURES = pd.concat([df_remaining, ZIP_FEATURES[cols_to_move]], axis=1)

ZIP_FEATURES

,ZIP,USPS_STATE,TYPE,CD_BUS_RATIO,CD_OTH_RATIO,CD_TOT_RATIO,COUNTY_FIPS,RES_RATIO_x,BUS_RATIO_x,OTH_RATIO_x,...,OTH_RATIO_y,TOT_RATIO_y,CONGRESSPERSON,STATE,District,CONGRESSPERSON_PARTY,Years_Served,FIPS_Code,CD,CD_RES_RATIO
0,01001,MA,Standard/PO BOX,1.0,1.0,1.0,25013,1.0,1.0,1.0,...,0.250000,0.403828,"Neal, Richard E.",Massachusetts,01,Democratic,House: 1989-Present,25,2501,1.0
1,45729,OH,Standard/PO BOX,1.0,1.0,1.0,39167,1.0,1.0,1.0,...,1.000000,0.749772,"Johnson, Bill",Ohio,06,Republican,House: 2011-Present,39,3906,1.0
2,45771,OH,Standard/PO BOX,1.0,1.0,1.0,39105,1.0,1.0,1.0,...,0.692308,0.788959,"Johnson, Bill",Ohio,06,Republican,House: 2011-Present,39,3906,1.0
3,45770,OH,Standard/PO BOX,1.0,0.0,1.0,39105,1.0,1.0,0.0,...,0.000000,0.924791,"Johnson, Bill",Ohio,06,Republican,House: 2011-Present,39,3906,1.0
4,45769,OH,Standard/PO BOX,1.0,1.0,1.0,39105,1.0,1.0,1.0,...,0.127660,0.440522,"Johnson, Bill",Ohio,06,Republican,House: 2011-Present,39,3906,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40766,99711,AK,Standard/PO BOX,NaN,NaN,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,"Young, Don",Alaska,00,Republican,House: 1973-2022,02,0200,NaN
40767,99716,AK,Standard/PO BOX,NaN,NaN,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,"Young, Don",Alaska,00,Republican,House: 1973-2022,02,0200,NaN
40768,99802,AK,Standard/PO BOX,NaN,NaN,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,"Young, Don",Alaska,00,Republican,House: 1973-2022,02,0200,NaN
40769,99803,AK,Standard/PO BOX,NaN,NaN,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,"Young, Don",Alaska,00,Republican,House: 1973-2022,02,0200,NaN


In [127]:
col_types = {
    'county_fips': str
}

mask_mandates_count = pd.read_excel('data/county_mask_mandate_data.xlsx', dtype=col_types)
mask_mandates_count = mask_mandates_count[['county_fips','county_conditions','state_conditions']]

mask_mandates_count['county_fips'] = mask_mandates_count['county_fips'].astype(str).str.zfill(5)

mask_mandates_count['county_conditions'] = mask_mandates_count.apply(
    lambda row: row['state_conditions'] if pd.isna(row['county_conditions']) and not pd.isna(row['state_conditions']) else row['county_conditions'],
    axis=1
)

mask_mandates_count['county_conditions'] = mask_mandates_count.apply(
    lambda row: "no mandate" if pd.isna(row['county_conditions']) else row['county_conditions'],
    axis=1
)

mask_mandates_count = mask_mandates_count.rename(columns = {
    'county_fips': 'COUNTY_FIPS'
})

print(mask_mandates_count.head())

ZIP_FEATURES = pd.merge(ZIP_FEATURES, mask_mandates_count, on = "COUNTY_FIPS", how= "left")

print(ZIP_FEATURES.head())

  COUNTY_FIPS  county_conditions   state_conditions
0       01067  all public places  all public places
1       01071  all public places  all public places
2       01093        retail only  all public places
3       01073  all public places  all public places
4       01099  all public places  all public places
     ZIP USPS_STATE             TYPE  CD_BUS_RATIO  CD_OTH_RATIO  \
0  01001         MA  Standard/PO BOX           1.0           1.0   
1  45729         OH  Standard/PO BOX           1.0           1.0   
2  45771         OH  Standard/PO BOX           1.0           1.0   
3  45770         OH  Standard/PO BOX           1.0           0.0   
4  45769         OH  Standard/PO BOX           1.0           1.0   

   CD_TOT_RATIO COUNTY_FIPS  RES_RATIO_x  BUS_RATIO_x  OTH_RATIO_x  ...  \
0           1.0       25013          1.0          1.0          1.0  ...   
1           1.0       39167          1.0          1.0          1.0  ...   
2           1.0       39105          1.0          1.0 

I have a pretty decent match rate of 95%. However, let's take a look at some of the non-matches and see if we can fill in the gaps

In [129]:
print(ZIP_FEATURES['CONGRESSPERSON'].isna().sum())
print(ZIP_FEATURES['CONGRESSPERSON'].isna().mean() * 100)

2732
6.700841284246155


In [130]:
print(ZIP_FEATURES['county_conditions'].isna().sum())
print(ZIP_FEATURES['county_conditions'].isna().mean() * 100)

790
1.9376517622820142


Below, we need to add a leading "0" to the FIPS code (FIPS codes are all 5 digits) \
These are cumulative mortality numbers (not weekly)

In [131]:
col_types = {
    'countyFIPS': str
}

usafacts_deaths = pd.read_csv('Data/Partisanship and Health Behavior/Data/covid_deaths_usafacts.csv', dtype=col_types)
usafacts_deaths['countyFIPS'] = usafacts_deaths['countyFIPS'].astype(str).str.zfill(5)

For this preliminary analysis, I will only use wave 2,4,6. \
Each wave's data was collected on 2020-06-17 - 2020-06-23, 2020-11-29 - 2020-12-16, 2021-05-12 - 2021-05-25. \
I will take covid mortality from the midpoint of the wave range. \
Wave 2 = 6/18-24. 2020 \
Wave 3 = 9/12-20, 2020 \
Wave 4 = 12/1-7, 2020 \
Wave 5 = 2/9-14, 2021 \
Wave 6 = 5/14-25, 2021

In [132]:
columns_to_keep = ['countyFIPS','County Name',
                   '2020-06-01','2020-06-15','2020-06-13','2020-06-20', #w2
                   '2020-09-01','2020-09-16','2020-09-09','2020-09-05', #w3
                   '2020-12-05','2020-12-04','2020-12-01','2020-11-28', #w4
                   '2021-02-12','2021-02-10','2021-02-05','2021-01-29', #w5
                   '2021-05-19','2021-05-17','2021-05-12','2021-05-05'] #w6

usafacts_deaths = usafacts_deaths[columns_to_keep]

new_column_names = [col if idx < 2 else 'COUNTY_COV_DEATHS_' + col.replace('-', '_') for idx, col in enumerate(columns_to_keep)]

usafacts_deaths.columns = new_column_names

usafacts_deaths = usafacts_deaths.rename(columns={
    'countyFIPS': 'COUNTY_FIPS'
})

usafacts_deaths.head()

,COUNTY_FIPS,County Name,COUNTY_COV_DEATHS_2020_06_01,COUNTY_COV_DEATHS_2020_06_15,COUNTY_COV_DEATHS_2020_06_13,COUNTY_COV_DEATHS_2020_06_20,COUNTY_COV_DEATHS_2020_09_01,COUNTY_COV_DEATHS_2020_09_16,COUNTY_COV_DEATHS_2020_09_09,COUNTY_COV_DEATHS_2020_09_05,...,COUNTY_COV_DEATHS_2020_12_01,COUNTY_COV_DEATHS_2020_11_28,COUNTY_COV_DEATHS_2021_02_12,COUNTY_COV_DEATHS_2021_02_10,COUNTY_COV_DEATHS_2021_02_05,COUNTY_COV_DEATHS_2021_01_29,COUNTY_COV_DEATHS_2021_05_19,COUNTY_COV_DEATHS_2021_05_17,COUNTY_COV_DEATHS_2021_05_12,COUNTY_COV_DEATHS_2021_05_05
0,00000,Statewide Unallocated,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,01001,Autauga County,5,6,6,9,24,24,24,24,...,41,41,84,79,76,69,108,108,108,108
2,01003,Baldwin County,9,9,9,9,42,47,46,46,...,137,98,251,239,234,224,310,310,309,307
3,01005,Barbour County,1,1,1,1,7,7,7,7,...,11,10,48,46,44,40,56,56,56,56
4,01007,Bibb County,1,1,1,1,8,9,9,9,...,18,17,57,54,52,51,64,64,64,63


Nearly all ZIP codes get a match (less than 1 percent don't get matched). \
If we end up going back to mortality rates (as of now (01/14/24) we're using prevalence rates. 

In [133]:
ZIP_FEATURES = pd.merge(ZIP_FEATURES, usafacts_deaths, on = "COUNTY_FIPS", how = "left")

print(ZIP_FEATURES['COUNTY_COV_DEATHS_2020_06_01'].isna().sum())
print(ZIP_FEATURES['COUNTY_COV_DEATHS_2020_06_01'].isna().mean() * 100)

664
1.6286085698167814


In [134]:
col_types1 = {
    'STATE': str,
    'COUNTY': str
}

col_types2 = {
    'County Code (FIPS)': str,
    'County Subdivision Code (FIPS)': str,
    'Place Code (FIPS)': str,
    'Consolidtated City Code (FIPS)': str,
    'State Code (FIPS)': str
}


census_pop = pd.read_csv('Data/Partisanship and Health Behavior/Data/co-est2021-alldata.csv', encoding='ISO-8859-1',
                        dtype = col_types1)
census_pop_FIPS = pd.read_excel('Data/Partisanship and Health Behavior/Data/all-geocodes-v2020.xlsx', 
                                dtype = col_types2)

census_pop_FIPS = census_pop_FIPS.rename(columns={
    'Area Name (including legal/statistical area description)': 'CTYNAME'
})

census_pop['COUNTY_FIPS'] = census_pop['STATE']+census_pop['COUNTY']

State_FIPS_Crosswalk = census_pop[['STATE','STNAME']]
State_FIPS_Crosswalk['STATE_FIPS'] = State_FIPS_Crosswalk['STATE'].astype(str)
State_FIPS_Crosswalk['STATE'] = State_FIPS_Crosswalk['STNAME']

State_FIPS_Crosswalk = State_FIPS_Crosswalk.drop(columns = ['STNAME'])

State_FIPS_Crosswalk = State_FIPS_Crosswalk.drop_duplicates(subset='STATE', keep='first')
State_FIPS_Crosswalk = State_FIPS_Crosswalk[pd.notna(State_FIPS_Crosswalk['STATE'])].reset_index(drop=True)

State_FIPS_Crosswalk.to_csv('data/STATE_FIPS_CROSSWALK.csv', index = False)
State_FIPS_Crosswalk.to_csv('data/Partisanship and Health Behavior/Data/STATE_FIPS_CROSSWALK.csv', index = False)

columns_to_keep = ['STATE','POPESTIMATE2020','DEATHS2020','POPESTIMATE2021','DEATHS2021','COUNTY_FIPS']

census_pop = census_pop[columns_to_keep]

census_pop = census_pop.rename(columns={
    'STATE': 'STATE_FIPS'
})

/var/folders/89/6bdxzk2j30v5n3wstywbcpg80000gn/T/ipykernel_41031/4051827355.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  State_FIPS_Crosswalk['STATE_FIPS'] = State_FIPS_Crosswalk['STATE'].astype(str)
/var/folders/89/6bdxzk2j30v5n3wstywbcpg80000gn/T/ipykernel_41031/4051827355.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  State_FIPS_Crosswalk['STATE'] = State_FIPS_Crosswalk['STNAME']


A crude mortality rate, not standardized or age adjusted, for this preliminary analysis \
I manually insert Puerto Rico' State FIPS code and District of Columbia to make this a clean merge across the board.

In [137]:
ZIP_FEATURES = pd.merge(ZIP_FEATURES, census_pop, on = "COUNTY_FIPS", how = "left")


ZIP_FEATURES['COV_County_CMR_2020_06_20'] = (ZIP_FEATURES['COUNTY_COV_DEATHS_2020_06_20'] / ZIP_FEATURES['POPESTIMATE2020']) * 1000
ZIP_FEATURES['COV_County_CMR_2020_12_09'] = (ZIP_FEATURES['COUNTY_COV_DEATHS_2020_12_05'] / ZIP_FEATURES['POPESTIMATE2020']) * 1000
ZIP_FEATURES['COV_County_CMR_2021_05_19'] = (ZIP_FEATURES['COUNTY_COV_DEATHS_2021_05_19'] / ZIP_FEATURES['POPESTIMATE2021']) * 1000
ZIP_FEATURES['STATE_FIPS'] = ZIP_FEATURES.apply(lambda row: 72 if row['STATE'] == "Puerto Rico" else row['STATE_FIPS'], axis=1)

print(ZIP_FEATURES['COV_County_CMR_2020_06_20'].isna().sum())
print(ZIP_FEATURES['COV_County_CMR_2020_06_20'].isna().mean() * 100)

672
1.6482303598145742


We have decided that prevalence is a better measure for predicting contact behavior than mortality \
I want these lags \
Wave 2 (6/18-24, 2020), midpoint: 06/21/2020, lag1: 06/14/2020, lag3: 06/07/2020 \
Wave 3 (9/12-20, 2020), midpoint: 09/16/2020, lag1: 09/09/2020, lag3: 09/02/2020 \
Wave 4 (12/1-7, 2020), midpoint: 12/04/2020, lag1: 11/27/2020, lag3: 11/20/2020 \
Wave 5 (2/9-14, 2021), midpoint: 02/11/2021, lag: 02/04/2021, lag3: 01/28/2021 \
Wave 6 (5/14-25, 2021), midpoint: 05/19/2021, lag: 05/12/2021, lag3: 05/05/2021 

In [139]:
col_types = {
    'FIPS': str,
    'iso2':str
}


prevalence_mid_2020 = pd.read_csv('Data/Partisanship and Health Behavior/Data/JH_Prevalence/time_series_covid19_confirmed_US_mid_2020.csv', 
                                  dtype = col_types)

columns_to_keep = ['FIPS','6/21/20','6/18/20','6/14/20','6/7/20']

prevalence_mid_2020 = prevalence_mid_2020[prevalence_mid_2020['iso2'] == 'US']
prevalence_mid_2020['FIPS'] = prevalence_mid_2020['FIPS'].apply(lambda x: str(x).rstrip('0').rstrip('.') if pd.notna(x) and '.' in str(x) else x)
prevalence_mid_2020['FIPS'] = prevalence_mid_2020['FIPS'].apply(lambda x: str(x).zfill(5))

prevalence_mid_2020 = prevalence_mid_2020[columns_to_keep]

#06/21/2020 - 06/14/2020
prevalence_mid_2020['wave_2_incidence'] = prevalence_mid_2020['6/21/20'] - prevalence_mid_2020['6/14/20']
#06/14/2020 - 06/07/2020
prevalence_mid_2020['wave_2_inc_week_lag'] = prevalence_mid_2020['6/14/20'] - prevalence_mid_2020['6/7/20']

prevalence_mid_2020.to_csv('data/wave_2_incidence.csv', index=False)

prevalence_late_2020 =pd.read_csv('Data/Partisanship and Health Behavior/Data/JH_Prevalence/time_series_covid19_confirmed_US_late_2020.csv', 
                                  dtype = col_types)

columns_to_keep = ['FIPS','9/16/20','9/12/20','9/9/20','9/2/20']

prevalence_late_2020 = prevalence_late_2020[prevalence_late_2020['iso2'] == 'US']
prevalence_late_2020['FIPS'] = prevalence_late_2020['FIPS'].apply(lambda x: str(x).rstrip('0').rstrip('.') if pd.notna(x) and '.' in str(x) else x)
prevalence_late_2020['FIPS'] = prevalence_late_2020['FIPS'].apply(lambda x: str(x).zfill(5))

prevalence_late_2020 = prevalence_late_2020[columns_to_keep]

#09/16/2020 - 09/09/2020
prevalence_late_2020['wave_3_incidence'] = prevalence_late_2020['9/16/20'] - prevalence_late_2020['9/9/20']
#09/09/2020 - 9/2/2020
prevalence_late_2020['wave_3_inc_week_lag'] = prevalence_late_2020['9/9/20'] - prevalence_late_2020['9/2/20']

prevalence_late_2020.to_csv('data/wave_3_incidence.csv', index=False)

prevalence_early_2021 = pd.read_csv('Data/Partisanship and Health Behavior/Data/JH_Prevalence/time_series_covid19_confirmed_US_early_2021.csv', 
                                  dtype = col_types)

columns_to_keep = ['FIPS','12/4/20','12/1/20','11/27/20','11/20/20']

prevalence_early_2021 = prevalence_early_2021[prevalence_early_2021['iso2'] == 'US']
prevalence_early_2021['FIPS'] = prevalence_early_2021['FIPS'].apply(lambda x: str(x).rstrip('0').rstrip('.') if pd.notna(x) and '.' in str(x) else x)
prevalence_early_2021['FIPS'] = prevalence_early_2021['FIPS'].apply(lambda x: str(x).zfill(5))

prevalence_early_2021 = prevalence_early_2021[columns_to_keep]

#12/4/20 - 11/27/20
prevalence_early_2021['wave_4_incidence'] = prevalence_early_2021['12/4/20'] - prevalence_early_2021['11/27/20']
#11/27/20 - 11/20/20
prevalence_early_2021['wave_4_inc_week_lag'] = prevalence_early_2021['11/27/20'] - prevalence_early_2021['11/20/20']

prevalence_early_2021.to_csv('data/wave_4_incidence.csv', index=False)

prevalence_mid_2021 = pd.read_csv('Data/Partisanship and Health Behavior/Data/JH_Prevalence/time_series_covid19_confirmed_US_mid_2021.csv', 
                                  dtype = col_types)

columns_to_keep = ['FIPS','2/11/21','2/9/21','2/4/21','1/28/21']

prevalence_mid_2021 = prevalence_mid_2021[prevalence_mid_2021['iso2'] == 'US']
prevalence_mid_2021['FIPS'] = prevalence_mid_2021['FIPS'].apply(lambda x: str(x).rstrip('0').rstrip('.') if pd.notna(x) and '.' in str(x) else x)
prevalence_mid_2021['FIPS'] = prevalence_mid_2021['FIPS'].apply(lambda x: str(x).zfill(5))

prevalence_mid_2021 = prevalence_mid_2021[columns_to_keep]

#02/11/2021 - 02/04/2021
prevalence_mid_2021['wave_5_incidence'] = prevalence_mid_2021['2/11/21'] - prevalence_mid_2021['2/4/21']
#2/4/2021 - 1/28/2021
prevalence_mid_2021['wave_5_inc_week_lag'] = prevalence_mid_2021['2/4/21'] - prevalence_mid_2021['1/28/21']

prevalence_mid_2021.to_csv('data/wave_5_incidence.csv', index=False)

prevalence_mid_2021_2 = pd.read_csv('Data/Partisanship and Health Behavior/Data/JH_Prevalence/time_series_covid19_confirmed_US_mid_2021.csv', 
                                  dtype = col_types)

colums_to_keep = ['FIPS','5/19/21','5/14/21','5/12/21','5/5/21']

prevalence_mid_2021_2 = prevalence_mid_2021_2[prevalence_mid_2021_2['iso2'] == 'US']
prevalence_mid_2021_2['FIPS'] = prevalence_mid_2021_2['FIPS'].apply(lambda x: str(x).rstrip('0').rstrip('.') if pd.notna(x) and '.' in str(x) else x)
prevalence_mid_2021_2['FIPS'] = prevalence_mid_2021_2['FIPS'].apply(lambda x: str(x).zfill(5))

prevalence_mid_2021_2 = prevalence_mid_2021_2[colums_to_keep]

#05/19/2021 - 05/12/2021
prevalence_mid_2021_2['wave_6_incidence'] = prevalence_mid_2021_2['5/19/21'] - prevalence_mid_2021_2['5/12/21']
#05/12/2021 - 05/5/2021
prevalence_mid_2021_2['wave_6_inc_week_lag'] = prevalence_mid_2021_2['5/12/21'] - prevalence_mid_2021_2['5/5/21']

prevalence_mid_2021_2.to_csv('data/wave_6_incidence.csv', index=False)

prevalence = pd.merge(prevalence_mid_2020, prevalence_late_2020, how = "left", on = "FIPS")
prevalence = pd.merge(prevalence, prevalence_early_2021, how = "left", on = "FIPS")
prevalence = pd.merge(prevalence, prevalence_mid_2021, how = "left", on = "FIPS")
prevalence = pd.merge(prevalence, prevalence_mid_2021_2, how = 'left', on = "FIPS")
prevalence.columns = [col if i == 0 else 'prev_' + col for i, col in enumerate(prevalence.columns)]

prevalence = prevalence.rename(columns={
    'FIPS':'COUNTY_FIPS'
})

prevalence

,COUNTY_FIPS,prev_6/21/20,prev_6/18/20,prev_6/14/20,prev_6/7/20,prev_wave_2_incidence,prev_wave_2_inc_week_lag,prev_9/16/20,prev_9/12/20,prev_9/9/20,...,prev_2/4/21,prev_1/28/21,prev_wave_5_incidence,prev_wave_5_inc_week_lag,prev_5/19/21,prev_5/14/21,prev_5/12/21,prev_5/5/21,prev_wave_6_incidence,prev_wave_6_inc_week_lag
0,01001,428,394,357,265,71,92,1619,1432,1398,...,5811,5554,159,257,7017,6971,6928,6914,89,14
1,01003,415,386,364,313,51,51,5003,4752,4639,...,18494,17779,466,715,21467,21290,21170,21035,297,135
2,01005,271,262,234,193,37,41,809,620,618,...,1989,1920,41,69,2324,2319,2314,2307,10,7
3,01007,124,118,104,77,20,27,612,576,564,...,2327,2271,50,56,2652,2630,2612,2604,40,8
4,01009,146,127,110,72,36,38,1487,1114,1082,...,5842,5612,113,230,6808,6750,6680,6635,128,45
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103241,00nan,666,588,502,387,164,115,3842,3712,3607,...,26422,25802,479,620,8991,8961,8939,8877,52,62
103242,00nan,666,588,502,387,164,115,3842,3712,3607,...,26422,25802,479,620,3552,3536,3516,3483,36,33
103243,00nan,666,588,502,387,164,115,3842,3712,3607,...,26422,25802,479,620,29041,28939,28891,28762,150,129
103244,00nan,666,588,502,387,164,115,3842,3712,3607,...,26422,25802,479,620,4116,4079,4060,4022,56,38


Next, I want to create the weekly incidence for each key date

#06/21/2020 - 06/14/2020
prevalence['wave_2_incidence'] = prevalence['prev_6/21/20'] - prevalence['prev_6/14/20']
#09/16/2020 - 09/09/2020
prevalence['wave_3_incidence'] = prevalence['prev_9/16/20'] - prevalence['prev_9/9/20']
#12/04/2020 - 11/27/2020
prevalence['wave_4_incidence'] = prevalence['prev_12/4/20'] - prevalence['prev_11/27/20']
#02/11/2021 - 02/04/2021
prevalence['wave_5_incidence'] = prevalence['prev_2/11/21'] - prevalence['prev_2/4/21']
#05/19/2021 - 05/12/2021
prevalence['wave_6_incidence'] = prevalence['prev_5/19/21'] - prevalence['prev_5/12/21']

below we turn these figures into rates based on population

In [140]:
ZIP_FEATURES = pd.merge(ZIP_FEATURES, prevalence, on = "COUNTY_FIPS", how = "left")

for col in ZIP_FEATURES.columns:
    # Check if the column name starts with 'prev_'
    if col.startswith('prev_'):
        # Extract the date part of the column name
        date_part = col.split('_', 1)[1]
        # Perform the calculation and create a new column
        ZIP_FEATURES['rate_prev_' + date_part] = (ZIP_FEATURES[col] / ZIP_FEATURES['POPESTIMATE2020']) * 1000
        
prev_columns = prevalence.columns
prev_columns = prev_columns[1:]
ZIP_FEATURES = ZIP_FEATURES.drop(columns=prev_columns)

print(ZIP_FEATURES['rate_prev_11/27/20'].isna().sum())
print(ZIP_FEATURES['rate_prev_11/27/20'].isna().mean() * 100)

672
1.6482303598145742


Below I insert data on urbanicity

In [ ]:
columns_to_keep = ['FIPS','RUCC_2013']

col_types = {
    'FIPS': str,
}

RUC_county_df = pd.read_csv('Data/Partisanship and Health Behavior/Data/ruralurbancodes2013.csv', dtype = col_types)

RUC_county_df = RUC_county_df[columns_to_keep]

RUC_county_df = RUC_county_df.rename(columns={
    'RUCC_2013': 'COUNTY_RUCC_2013',
    'FIPS':'COUNTY_FIPS'
})

RUC_county_df['COUNTY_FIPS'] = RUC_county_df['COUNTY_FIPS'].astype(str)

ZIP_FEATURES = pd.merge(ZIP_FEATURES, RUC_county_df, on = "COUNTY_FIPS", how = "left")

Next, I want to control for the severity of the lockdowns according to Oxford's tracker

In [ ]:
State_Restrictions_df = pd.read_csv('Data/Partisanship and Health Behavior/Data/OxCGRT_USA_differentiated_withnotes_2020.csv')
State_Restrictions_df['Date'] = State_Restrictions_df['Date'].astype(str)
State_Restrictions_df = State_Restrictions_df[State_Restrictions_df['Date'].isin(['20200617', '20201129'])]
columns_to_keep = ['RegionName','GovernmentResponseIndex_SimpleAverage','Date']
State_Restrictions_df = State_Restrictions_df[columns_to_keep]

State_Restrictions_df = State_Restrictions_df.rename(columns={
    'RegionName': 'STATE',
    'GovernmentResponseIndex_SimpleAverage':'State_Government_Response_Index'
})

State_Restrictions_df

In [ ]:
State_Restrictions_df['State_Government_Response_Index_06172020'] = State_Restrictions_df.apply(
    lambda row: row['State_Government_Response_Index'] if row['Date'] == "20200617" else "0", axis=1
)

State_Restrictions_df['State_Government_Response_Index_06172020'] = State_Restrictions_df.apply(
    lambda row: row['State_Government_Response_Index'] if row['State_Government_Response_Index_06172020'] == "0" else row['State_Government_Response_Index_06172020'], axis=1
)

State_Restrictions_df['State_Government_Response_Index_11192020'] = State_Restrictions_df.apply(
    lambda row: row['State_Government_Response_Index'] if row['Date'] == "20201129" else "0", axis=1
)

State_Restrictions_df['State_Government_Response_Index_11192020'] = State_Restrictions_df.apply(
    lambda row: row['State_Government_Response_Index'] if row['State_Government_Response_Index_11192020'] == "0" else row['State_Government_Response_Index_11192020'], axis=1
)

State_Restrictions_df

In [ ]:
ZIP_FEATURES = pd.merge(ZIP_FEATURES, State_Restrictions_df, on = "STATE", how = "left")

ZIP_FEATURES

Next, I want to read in the county level presidential data and generate two variables \
1. For which president did the county overall vote for?
2. What percentage of the county voted for which president?

note: I manually added in DC because it was not in the dataset

In [ ]:
col_types = {
    'county_fips': str,
}
county_pres = pd.read_csv('Data/Partisanship and Health Behavior/Data/countypres_2000-2020.csv', dtype = col_types)
county_pres = county_pres[county_pres['year'] == 2020].reset_index(drop=True)

county_pres['party_fips'] = county_pres['party'].astype(str) + county_pres['county_fips'].astype(str)

#there was an issue where there were multiple types of votes on each line for the same person
total_votes_by_party = county_pres.groupby('party_fips')['candidatevotes'].sum().reset_index()
total_votes_by_party.rename(columns={'candidatevotes': 'total_candidatevotes'}, inplace=True)

county_pres = county_pres.merge(total_votes_by_party, on='party_fips', how='left')

county_pres['percentage_won'] = (county_pres['total_candidatevotes'] / county_pres['totalvotes'])*100

# Group by 'countyfips' and find the index of the row with the largest 'percentage_won' in each group
county_pres_wide = county_pres.groupby('county_fips')['percentage_won'].idxmax()

# Select rows from the original DataFrame corresponding to these indices
county_pres_wide = county_pres.loc[county_pres_wide]

county_pres_wide = county_pres_wide.rename(columns={
    'candidate': 'winning_pres_candidate',
    'party':'winning_pres_party',
    'percentage_won': 'winning_pres_candidate_percentage'
})

#merge in the dems
dem_county_pres = county_pres[county_pres['party'] == 'DEMOCRAT']

dem_county_pres.rename(columns={'percentage_won': 'biden_percentage_won'}, inplace=True)

dem_county_pres = dem_county_pres[['biden_percentage_won','county_fips']]

county_pres_wide = county_pres_wide.merge(dem_county_pres, on='county_fips', how='left')

#merge in the reps
rep_county_pres = county_pres[county_pres['party'] == 'REPUBLICAN']

rep_county_pres.rename(columns={'percentage_won': 'trump_percentage_won'}, inplace=True)

rep_county_pres = rep_county_pres[['trump_percentage_won','county_fips']]

county_pres_wide = county_pres_wide.merge(rep_county_pres, on='county_fips', how='left')

#keeping only the columns I want
county_pres_wide = county_pres_wide.drop(columns =['year','state','state_po','county_name','office','candidatevotes','totalvotes','version','mode','party_fips'])

county_pres_wide

In [ ]:
county_pres_wide = county_pres_wide.rename(columns={
    'county_fips': 'COUNTY_FIPS'
})

ZIP_FEATURES = pd.merge(ZIP_FEATURES, county_pres_wide, on = "COUNTY_FIPS", how = "left")

ZIP_FEATURES

Next, for the sake of consistency, I want to get data on the percentage of the CD that was won by a democrat and Republican, respectively. 

However, I need a clean crosswalk (as this one is pulling out several duplicates)

In [ ]:
col_types = {
    'DISTRICT': str,
}

county_CD_share = pd.read_excel('Data/Partisanship and Health Behavior/Data/federalelections2020.xlsx', 
                                sheet_name='13_US_House_Results_State', 
                                dtype=col_types)

county_CD_share = county_CD_share.rename(columns={
    'DISTRICT': 'CD_DISTRICT'
})

county_CD_share = county_CD_share[pd.notna(county_CD_share['GENERAL %'])].reset_index(drop=True)

county_CD_share = pd.merge(county_CD_share, State_FIPS_Crosswalk, how='left', on="STATE")

county_CD_share['STATE_FIPS'] = county_CD_share['STATE_FIPS'].astype(str)
county_CD_share['CD_DISTRICT'] = county_CD_share['CD_DISTRICT'].astype(str)

county_CD_share['STATE_FIPS'] = county_CD_share['STATE_FIPS'].apply(lambda x: '0' + x if len(x) == 1 else x)
county_CD_share['CD_DISTRICT'] = county_CD_share['CD_DISTRICT'].apply(lambda x: '0' + x if len(x) == 1 else x)

county_CD_share['CD'] = county_CD_share['STATE_FIPS'] + county_CD_share['CD_DISTRICT']

county_CD_share.to_csv('data/CD_COUNTY_PERCENT_WON.csv')

D_county_CD_share = county_CD_share[county_CD_share['PARTY'].str.contains("D")].reset_index(drop=True)
D_county_CD_share = D_county_CD_share[['GENERAL %','CD']]

D_county_CD_share = D_county_CD_share.rename(columns={
    'GENERAL %': 'CD_PERCENT_DEMOCRAT'
})

R_county_CD_share = county_CD_share[county_CD_share['PARTY'].str.contains("R")].reset_index(drop=True)
R_county_CD_share = R_county_CD_share[['GENERAL %','CD']]

R_county_CD_share = R_county_CD_share.rename(columns={
    'GENERAL %': 'CD_PERCENT_REPUBLICAN'
})

D_county_CD_share = D_county_CD_share[D_county_CD_share['CD'].notna()]
R_county_CD_share = R_county_CD_share[R_county_CD_share['CD'].notna()]

ZIP_FEATURES = pd.merge(ZIP_FEATURES, D_county_CD_share, how = 'left', on = "CD")
ZIP_FEATURES = pd.merge(ZIP_FEATURES, R_county_CD_share, how = 'left', on = "CD")

print(ZIP_FEATURES['CD_PERCENT_REPUBLICAN'].isna().sum())
print(ZIP_FEATURES['CD_PERCENT_REPUBLICAN'].isna().mean() * 100)

#manually adjusting these rows that received 0 votes in democratic counties
ZIP_FEATURES['CD_PERCENT_REPUBLICAN'] = ZIP_FEATURES.apply(
    lambda row: 0 if pd.notna(row['CD_PERCENT_DEMOCRAT']) and pd.isna(row['CD_PERCENT_REPUBLICAN']) else row['CD_PERCENT_REPUBLICAN'],
    axis=1
)

#now in republican counties
ZIP_FEATURES['CD_PERCENT_DEMOCRAT'] = ZIP_FEATURES.apply(
    lambda row: 0 if row['CD_PERCENT_REPUBLICAN'] != 0 and pd.notna(row['CD_PERCENT_REPUBLICAN']) and pd.isna(row['CD_PERCENT_DEMOCRAT']) else row['CD_PERCENT_DEMOCRAT'],
    axis=1
)

ZIP_FEATURES['CD'] = ZIP_FEATURES.apply(lambda row: 0 if row['CD'] == "1225" else row['CD'],
                                                        axis=1)

ZIP_FEATURES['CD'] = ZIP_FEATURES.apply(lambda row: 100 if row['CD'] == "1225" else row['CD'],
                                                        axis=1)

print(ZIP_FEATURES['CD_PERCENT_REPUBLICAN'].isna().sum())
print(ZIP_FEATURES['CD_PERCENT_REPUBLICAN'].isna().mean() * 100)

Lastly, I want to merge in demographic data by CD

In [ ]:
col_types = {
    'GEO_ID': str,
}

CD_demographics = pd.read_csv('data/ACSDP5Y2020.DP05-Data.csv', dtype=col_types, header=1)

CD_demographics = CD_demographics[['Geography', 'Geographic Area Name', 'Estimate!!SEX AND AGE!!Total population', 
                                   'Estimate!!SEX AND AGE!!Total population!!Female', 'Estimate!!SEX AND AGE!!Total population!!Sex ratio (males per 100 females)',
                                  'Estimate!!SEX AND AGE!!Total population!!Under 5 years', 'Estimate!!SEX AND AGE!!Total population!!65 to 74 years',
                                  'Estimate!!SEX AND AGE!!Total population!!75 to 84 years','Estimate!!SEX AND AGE!!Total population!!85 years and over',
                                  'Estimate!!SEX AND AGE!!Total population!!Median age (years)','Estimate!!RACE!!Total population',
                                  'Estimate!!RACE!!Total population!!One race','Estimate!!RACE!!Total population!!Two or more races',
                                  'Estimate!!RACE!!Total population!!One race!!White','Estimate!!RACE!!Total population!!One race!!Black or African American',
                                  'Estimate!!RACE!!Total population!!One race!!American Indian and Alaska Native','Estimate!!RACE!!Total population!!One race!!Asian',
                                  'Estimate!!RACE!!Total population!!One race!!Asian!!Asian Indian','Margin of Error!!Race alone or in combination with one or more other races!!Total population!!White',
                                  'Estimate!!Race alone or in combination with one or more other races!!Total population!!Black or African American',
                                  'Estimate!!Race alone or in combination with one or more other races!!Total population!!American Indian and Alaska Native',
                                  'Estimate!!Race alone or in combination with one or more other races!!Total population!!Asian',
                                  'Estimate!!HISPANIC OR LATINO AND RACE!!Total population',
                                  'Estimate!!HISPANIC OR LATINO AND RACE!!Total population!!Hispanic or Latino (of any race)',
                                  'Estimate!!HISPANIC OR LATINO AND RACE!!Total population!!Not Hispanic or Latino',
                                  'Percent!!SEX AND AGE!!Total population!!65 years and over',
                                  'Percent!!RACE!!Total population!!One race','Percent!!RACE!!Total population!!Two or more races',
                                  'Percent!!RACE!!Total population!!One race!!White','Percent!!RACE!!Total population!!One race!!Black or African American',
                                  'Percent!!RACE!!Total population!!One race!!American Indian and Alaska Native','Percent!!RACE!!Total population!!One race!!Asian',
                                  'Percent!!RACE!!Total population!!One race!!Asian!!Asian Indian','Percent!!Race alone or in combination with one or more other races!!Total population!!White',
                                  'Percent!!Race alone or in combination with one or more other races!!Total population!!Black or African American',
                                  'Percent!!Race alone or in combination with one or more other races!!Total population!!American Indian and Alaska Native',
                                  'Percent!!Race alone or in combination with one or more other races!!Total population!!Asian',
                                  'Percent!!HISPANIC OR LATINO AND RACE!!Total population!!Hispanic or Latino (of any race)',
                                  'Percent!!HISPANIC OR LATINO AND RACE!!Total population!!Not Hispanic or Latino']]

In [ ]:
ZIP_FEATURES.to_csv('data/ZIP_Features.csv', index = False)

In [ ]:
ZIP_FEATURES.to_csv('data/Partisanship and Health Behavior/Data/ZIP_Features.csv', index = False)

Why is resp_educ and education different?

Now I'll merge to the BICS file

In [ ]:
col_types = {
    'resp_zip': str,
    'political_party': str
}

df_w1 = pd.read_csv('/Users/chrissoria/Documents/Research/BICS/data/national_wave1_unweighted.csv', dtype = col_types)
df_w2 = pd.read_csv('/Users/chrissoria/Documents/Research/BICS/data/national_wave2_unweighted.csv', dtype = col_types)
df_w3 = pd.read_csv('/Users/chrissoria/Documents/Research/BICS/data/national_wave3_unweighted.csv', dtype = col_types)
df_w4 = pd.read_csv('/Users/chrissoria/Documents/Research/BICS/data/national_wave4_unweighted.csv', dtype = col_types)
df_w5 = pd.read_csv('/Users/chrissoria/Documents/Research/BICS/data/national_wave5_unweighted.csv', dtype = col_types)
df_w6 = pd.read_csv('/Users/chrissoria/Documents/Research/BICS/data/national_wave6_unweighted.csv', dtype = col_types)

In [ ]:
if "resp_employ" in df_w2.columns:
    df_w2["resp_employ"] = df_w2["resp_employ"].astype(str)

if "resp_employ" in df_w3.columns:
    df_w3["resp_employ"] = df_w3["resp_employ"].astype(str)    
    
if "resp_employ" in df_w4.columns:
    df_w4["resp_employ"] = df_w4["resp_employ"].astype(str)
    
if "resp_employ" in df_w5.columns:
    df_w5["resp_employ"] = df_w5["resp_employ"].astype(str)

if "resp_employ" in df_w6.columns:
    df_w6["resp_employ"] = df_w6["resp_employ"].astype(str)
    
df_w2['resp_occupation'] = df_w2['resp_occupation'].astype(str)
df_w4['resp_occupation'] = df_w4['resp_occupation'].astype(str)

df_combined = pd.concat([df_w2, df_w3, df_w4, df_w5, df_w6], ignore_index=True)

Below I'm converting the waves into date ranges

In [ ]:
df_combined['data_collected_dates'] = df_combined.apply(lambda row: "June 18-24, '20" if row['wave'] == 2
                                                       else "September 12-20, '20'" if row['wave'] == 3
                                                       else "December 1-7, '20'" if row['wave'] == 4
                                                       else "February 9-14, '21'" if row['wave'] == 5
                                                       else "May 14-25, '21'" if row['wave'] == 6
                                                        else np.nan, axis = 1)

In [ ]:
print(df_combined['data_collected_dates'].value_counts())
print(df_combined['wave'].value_counts())

We're missing a "conservative" but we have two moderates?

In [ ]:
political_view_mapping = {
    'Extremely conservative': 1,
    'Slightly conservative': 2,
    'Moderate': 3,
    'Middle of the road': 3,
    'Slightly liberal': 4,
    'Liberal': 5,
    'Extremely liberal': 6
}

# Creating a new variable 'political_view_numeric' by mapping the 'political_view' column using the defined mapping
df_combined['political_view_numeric'] = df_combined['political_view'].replace(political_view_mapping)

df_combined['political_view'].value_counts()

In [ ]:
#education variable
def categorize_education(row):
    if row['resp_educ'] == "Less than high school degree":
        return "Less than high school"
    elif row['resp_educ'] in ["High school graduate (high school diploma or equivalent including GED)", "Some college but no degree", "Associate degree in college (2-year)"]:
        return "High school graduate"
    elif row['resp_educ'] in ["Bachelor's degree in college (4-year)", "Master's degree", "Doctoral degree", "Professional degree (JD, MD)"]:
        return "College graduate and above"
    else:
        return "Unknown"

df_combined['educ_group'] = df_combined.apply(categorize_education, axis=1)

df_combined['educ_group'].value_counts(dropna=False)

race_columns = [col for col in df_combined.columns if col.startswith('resp_race_')]
df_combined[race_columns] = df_combined[race_columns].fillna(0)

#race variable
def categorize_race(row):
    if row['resp_race_1'] == "White":
        return "White"
    elif row['resp_race_2'] == "Black or African American":
        return "Black"
    elif row['resp_race_4'] == "Asian":
        return "Asian"
    else:
        return "Other / Mixed"

df_combined['r_race'] = df_combined.apply(categorize_race, axis=1)

print(df_combined['r_race'].value_counts(dropna=False))

#employment variable
employment_columns = [col for col in df_combined.columns if col.startswith('resp_employ_')]
df_combined[employment_columns] = df_combined[employment_columns].fillna(0)

# Create the 'r_working' var based whether reported
df_combined['r_working'] = np.where(df_combined['industry'] == "I don't work", "Not Working", "Working")

print(df_combined['r_working'].value_counts(dropna=False))

dependent variable for concern

In [ ]:
df_combined['binary_concern'] = df_combined['covid19_concern'].apply(
    lambda x: 1 if x in ["Somewhat concerned", "Very concerned"] else (0 if pd.notna(x) else None)
)

df_combined['binary_concern_strong'] = df_combined['covid19_concern'].apply(
    lambda x: 1 if x == "Very concerned" else (0 if pd.notna(x) else None)
)

df_combined['contact_reduction'] = df_combined['covid19_f2fchange'].apply(
    lambda x: 1 if x == "I have greatly reduced face-to-face interaction with others" else (0 if pd.notna(x) else None)
)

reading in the alter files

In [ ]:
df_w2_nonhhalters = pd.read_csv("/Users/chrissoria/documents/research/BICs/data/national_alters_nonhh_wave2_unweighted.csv")
df_w3_nonhhalters = pd.read_csv("/Users/chrissoria/documents/research/BICs/data/national_alters_nonhh_wave3_unweighted.csv")
df_w4_nonhhalters = pd.read_csv("/Users/chrissoria/documents/research/BICs/data/national_alters_nonhh_wave4_unweighted.csv")
df_w5_nonhhalters = pd.read_csv("/Users/chrissoria/documents/research/BICs/data/national_alters_nonhh_wave5_unweighted.csv")
df_w6_nonhhalters = pd.read_csv("/Users/chrissoria/documents/research/BICs/data/national_alters_nonhh_wave6_unweighted.csv")

df_alters_combined = pd.concat([df_w2_nonhhalters, df_w3_nonhhalters, df_w4_nonhhalters, df_w5_nonhhalters, df_w6_nonhhalters], ignore_index=True)

In [ ]:
# Create binary indicators for mask, gloves, and other protective equipment usage
df_alters_combined['Mask_Used'] = (
    ((df_alters_combined['protection_1'] == "Wear a face mask") |
    (df_alters_combined['protection_2'] == "Wear a face mask") |
    (df_alters_combined['protection_3'] == "Wear a face mask") |
    (df_alters_combined['protection_4'] == "Wear a face mask")).astype(int)
)

df_alters_combined['Gloves_Used'] = (
    ((df_alters_combined['protection_1'] == "Wear gloves") |
    (df_alters_combined['protection_2'] == "Wear gloves") |
    (df_alters_combined['protection_3'] == "Wear gloves") |
    (df_alters_combined['protection_4'] == "Wear gloves")).astype(int)
)

df_alters_combined['Other_Protective_Equipment_Used'] = (
    ((df_alters_combined['protection_1'] == "Wear other protective equipment") |
    (df_alters_combined['protection_2'] == "Wear other protective equipment") |
    (df_alters_combined['protection_3'] == "Wear other protective equipment") |
    (df_alters_combined['protection_4'] == "Wear other protective equipment")).astype(int)
)

# Aggregate these indicators at the 'rid' level
df_aggregated = df_alters_combined.groupby('rid').agg(
    Total_Masks_Used=('Mask_Used', 'sum'),
    Total_Gloves_Used=('Gloves_Used', 'sum'),
    Total_Other_Protective_Equipment_Used=('Other_Protective_Equipment_Used', 'sum'),
    Contacts=('rid', 'size')
).reset_index()

# Normalize the counts by the number of contacts
df_aggregated['Norm_Masks_Used'] = df_aggregated['Total_Masks_Used'] / df_aggregated['Contacts']
df_aggregated['Norm_Gloves_Used'] = df_aggregated['Total_Gloves_Used'] / df_aggregated['Contacts']
df_aggregated['Norm_Other_Protective_Equipment_Used'] = df_aggregated['Total_Other_Protective_Equipment_Used'] / df_aggregated['Contacts']

# Calculate non-weighted and weighted safety indices
df_aggregated['Non_Weighted_Safety_Index'] = (
    df_aggregated['Norm_Masks_Used'] + 
    df_aggregated['Norm_Gloves_Used'] + 
    df_aggregated['Norm_Other_Protective_Equipment_Used']
) / 3

df_aggregated['Weighted_Safety_Index'] = (
    df_aggregated['Norm_Masks_Used'] + 
    df_aggregated['Norm_Gloves_Used'] + 
    df_aggregated['Norm_Other_Protective_Equipment_Used']
) / (3 * df_aggregated['Contacts'])

In [ ]:
df_combined_contacts = pd.merge(df_combined, df_aggregated, on = "rid", how = "left")

In [ ]:
df_combined_contacts['wave'].value_counts()

In [ ]:
df_combined_contacts['political_party'].value_counts()

In [ ]:
df_combined_contacts.to_csv('data/BICS_ego_alters_merged.csv', index = False)

In [ ]:
col_types = {
    'ZIP': str,
    'CD':str,
    'COUNTY_FIPS':str,
    'STATE_FIPS':str,
    'CONGRESSPERSON_PARTY': str
}

ZIP_FEATURES = pd.read_csv('/Users/chrissoria/Documents/Research/BICS_Political_Polarization/data/ZIP_Features.csv',
                          dtype=col_types)



In [ ]:
col_types = {
    'resp_zip': str,
    'political_party': str
}

all_waves = df_combined_contacts

all_waves = all_waves.rename(columns={
    'resp_zip': 'ZIP'
})

all_waves['ZIP'] = all_waves['ZIP'].str.strip()

columns_to_keep = ['ResponseId','StartDate','resp_yob','resp_sex','resp_hispanic','r_race','resp_nativity','ZIP',
                  'resp_hhsize','r_working','resp_occupation','lefthome_num','num_cc_nonhh',
                  'lefthome_where_1','lefthome_where_2','lefthome_where_3','lefthome_where_4','lefthome_where_10',
                  'lefthome_where_8','lefthome_where_9','lefthome_where_5','lefthome_where_11','lefthome_where_6',
                  'lefthome_where_7','inet_freq','socmedia_use','covid19_familiar','covid19_concern',
                  'covid19_f2fchange','covid19_reduceOK','policy_sip','age','hhi','political_party','political_view',
                  'industry','health_insurance','interview_date','wave','agecat','city','covid19_vax','covid19_whynot_vax',
                  'Non_Weighted_Safety_Index','Weighted_Safety_Index','Norm_Masks_Used','educ_group','contact_reduction',
                  'binary_concern','binary_concern_strong','resp_educ','resp_sex','Contacts','Non_Weighted_Safety_Index',
                  'Weighted_Safety_Index','data_collected_dates']
all_waves = all_waves[columns_to_keep]

all_waves['political_party'].value_counts()

How many people have a ZIP code? ALL

what percentage of these are 5 digit zip codes? A lot of these are four digit zip codes. There are so few of them it's not worth trying to figure out where they are (4 digits is enough to tell me something about their localility)

In [ ]:
BICS_ZIP_FEATURES = pd.merge(all_waves, ZIP_FEATURES, on = "ZIP", how = "left")
BICS_ZIP_FEATURES = BICS_ZIP_FEATURES.drop_duplicates(subset='ResponseId', keep='first')
BICS_ZIP_FEATURES

In [ ]:
print(BICS_ZIP_FEATURES['ZIP'].isna().sum())
print(BICS_ZIP_FEATURES['ZIP'].isna().mean() *100)
print((BICS_ZIP_FEATURES['ZIP'].apply(lambda x: len(str(x)) == 5).mean()) * 100)

In [ ]:
BICS_ZIP_FEATURES['junk_zip'] = BICS_ZIP_FEATURES['ZIP'].apply(lambda x: 1 if len(str(x)) == 5 else 0)
BICS_junk_zips = BICS_ZIP_FEATURES[BICS_ZIP_FEATURES['junk_zip'] == 0]
BICS_junk_zips

How many zip codes are being matched? \
currently I'm getting, 33 percent large percent missing. \
Fixed the issue by reading in ZIPS as string at the top now only .8788 don't match 

In [ ]:
print(BICS_ZIP_FEATURES['USPS_STATE'].isna().sum())
print(BICS_ZIP_FEATURES['USPS_STATE'].isna().mean() *100)

what about school closures? \
About 9 percent of people have a missing school matched to their ZIP

In [ ]:
print(BICS_ZIP_FEATURES['earlyfall2020_school_closures'].isna().sum())
print(BICS_ZIP_FEATURES['earlyfall2020_school_closures'].isna().mean() *100)

print(BICS_ZIP_FEATURES['fall2020_school_closures'].isna().sum())
print(BICS_ZIP_FEATURES['fall2020_school_closures'].isna().mean() *100)

print(BICS_ZIP_FEATURES['spring2021_school_closures'].isna().sum())
print(BICS_ZIP_FEATURES['spring2021_school_closures'].isna().mean() *100)

In [ ]:
BICS_ZIP_FEATURES['political_party_to_CD'] = BICS_ZIP_FEATURES.apply(
    lambda row: str(row['political_party']) + " in " + str(row['CONGRESSPERSON_PARTY']) + " led CD", axis=1
)

# Print the updated DataFrame to see if it matches
print(BICS_ZIP_FEATURES[['political_party', 'CONGRESSPERSON_PARTY', 'political_party_to_CD']])

In [ ]:
BICS_ZIP_FEATURES['political_party'].value_counts()

In [ ]:
BICS_ZIP_FEATURES['CONGRESSPERSON_PARTY'] = BICS_ZIP_FEATURES['CONGRESSPERSON_PARTY'].replace("Democratic", "Democrat")
BICS_ZIP_FEATURES['In_Opposing_Party_CD'] = ((BICS_ZIP_FEATURES['political_party'] != BICS_ZIP_FEATURES['CONGRESSPERSON_PARTY']) & ~pd.isna(BICS_ZIP_FEATURES['CONGRESSPERSON_PARTY'])).astype(int)
BICS_ZIP_FEATURES['Independent'] = (BICS_ZIP_FEATURES['political_party'] == "Independent").astype(int)
BICS_ZIP_FEATURES['Vaccinated'] = BICS_ZIP_FEATURES['covid19_vax'].apply(
    lambda x: 1 if x == "Yes, I have received at least one dose of a vaccine" else
              (0 if x == "No, I have not" else np.nan)
)

Does it make sense to drop Independents? \
They're kind of in their own cateogory

In [ ]:
print(BICS_ZIP_FEATURES['In_Opposing_Party_CD'].value_counts())
print(BICS_ZIP_FEATURES['political_party'].value_counts())
print(BICS_ZIP_FEATURES['CONGRESSPERSON_PARTY'].value_counts())

Below I convert the raw numbers to categories for Trump and Biden share

In [ ]:
BICS_ZIP_FEATURES['Categorical_Trump_County_Share'] = BICS_ZIP_FEATURES['trump_percentage_won'].apply(lambda x:
    "Less Than a Third" if x < 33.1 else
    "Less than Two Thirds Greater Than One Third" if x < 66.1 else
    "Greater than or Equal to Two thirds" if x >= 66.1 else pd.NA
)

BICS_ZIP_FEATURES['Categorical_Biden_County_Share'] = BICS_ZIP_FEATURES['biden_percentage_won'].apply(lambda x:
    "Less Than a Third" if x < 33.1 else
    "Less than Two Thirds Greater Than One Third" if x < 66.1 else
    "Greater than or Equal to Two thirds" if x >= 66.1 else pd.NA
)

BICS_ZIP_FEATURES['Percentage_Trump_Greater_Than_Two_Thirds'] = BICS_ZIP_FEATURES['trump_percentage_won'].apply(lambda x:
    1 if x >= 66.1 else
    0 if x < 66.1 else pd.NA
)

BICS_ZIP_FEATURES['Percentage_Trump_Less_Than_Two_Thirds'] = BICS_ZIP_FEATURES['trump_percentage_won'].apply(lambda x:
    1 if x < 66.1 else
    0 if x <= 66.1 else pd.NA
)

BICS_ZIP_FEATURES['Percentage_Biden_Greater_Than_Two_Thirds'] = BICS_ZIP_FEATURES['biden_percentage_won'].apply(lambda x:
    1 if x >= 66.1 else
    0 if x < 66.1 else pd.NA
)

BICS_ZIP_FEATURES['Percentage_Biden_Less_Than_Two_Thirds'] = BICS_ZIP_FEATURES['biden_percentage_won'].apply(lambda x:
    1 if x < 66.1 else
    0 if x <= 66.1 else pd.NA
)

Next, I use the same logic to create the CD categoricals

In [ ]:
BICS_ZIP_FEATURES['Categorical_Repub_CD_County_Share'] = BICS_ZIP_FEATURES['CD_PERCENT_REPUBLICAN'].apply(lambda x:
    "Less Than a Quarter" if x < .25 else
    "Two Quarters" if x < .50 and x >.25 else
    "Three Quarters" if x < .75 and x > .50 else
    "Greater than or Equal to Three Quarters" if x >= .75 else pd.NA
)

BICS_ZIP_FEATURES['Categorical_Dem_CD_County_Share'] = BICS_ZIP_FEATURES['CD_PERCENT_DEMOCRAT'].apply(lambda x:
    "Less Than a Quarter" if x < .25 else
    "Two Quarters" if x < .50 else
    "Three Quarters" if x < .75 else
    "Greater than or Equal to Three Quarters" if x >= .75 else pd.NA
)

BICS_ZIP_FEATURES['Categorical_Repub_CD_County_Share'].value_counts()

Next, let's add a variable that treats urban and rural as a binary. \
1 = Metro - Counties in metro areas of 1 million population or more \
2 = Metro - Counties in metro areas of 250,000 to 1 million population \         
3 = Metro - Counties in metro areas of fewer than 250,000 population \
4 = Nonmetro - Urban population of 20,000 or more, adjacent to a metro area \
5 = Nonmetro - Urban population of 20,000 or more, not adjacent to a metro area \
6 = Nonmetro - Urban population of 2,500 to 19,999, adjacent to a metro area \
7 = Nonmetro - Urban population of 2,500 to 19,999, not adjacent to a metro area \
8 = Nonmetro - Completely rural or less than 2,500 urban population, adjacent to a metro area \
9 = Nonmetro - Completely rural or less than 2,500 urban population, not adjacent to a metro area

In [ ]:
BICS_ZIP_FEATURES['COUNT_RUCC_CAT'] = BICS_ZIP_FEATURES['COUNTY_RUCC_2013'].apply(
    lambda x: "Urban Metro" if x in [1, 2] else
              "Urban Nonmetro" if x in list(range(3, 8)) else 
              "Rural" if x in [8, 9] else np.nan
)

BICS_ZIP_FEATURES['COUNT_RUCC_BINARY'] = BICS_ZIP_FEATURES['COUNTY_RUCC_2013'].apply(
    lambda x: "Urban" if x in list(range(1, 8)) else  
              "Rural" if x in [8, 9] else np.nan
)

Dopping all non-national values

In [ ]:
BICS_ZIP_FEATURES['city'].value_counts()

BICS_ZIP_FEATURES = BICS_ZIP_FEATURES[BICS_ZIP_FEATURES['city'] == "National"]

In [ ]:
BICS_ZIP_FEATURES.to_csv('data/BICS_ZIP_Features.csv',index=False)

In [ ]:
BICS_ZIP_FEATURES.to_csv('data/Partisanship and Health Behavior/Data/BICS_ZIP_Features.csv',index=False)